In [ ]:
#Connecting to google drive
from google.colab import drive 
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd gdrive/My Drive/RS_Challenge/RSLib
%ls

/content/gdrive/My Drive/RS_Challenge/RSLib
RecSys_Course_AT_PoliMi-master/


In [ ]:
!ls */*.py

RecSys_Course_AT_PoliMi-master/cython_example_FunkSVD.py
RecSys_Course_AT_PoliMi-master/cython_example_SLIM_MSE.py
RecSys_Course_AT_PoliMi-master/run_all_algorithms.py
RecSys_Course_AT_PoliMi-master/run_compile_all_cython.py
RecSys_Course_AT_PoliMi-master/run_hyperparameter_search.py
RecSys_Course_AT_PoliMi-master/run_test_recommenders.py


In [ ]:
#Importing course's repositories
import sys
sys.path.append('/content/gdrive/MyDrive/RS_Challenge/RSLib/RecSys_Course_AT_PoliMi-master')

In [ ]:
#To use skopt
! pip install --quiet lightgbm==2.2.3 scikit-optimize==0.9.0 neptune-client==0.4.132 neptune-contrib[monitoring]==0.25.0

     |████████████████████████████████| 100 kB 6.9 MB/s 
     |████████████████████████████████| 110 kB 43.2 MB/s 
     |████████████████████████████████| 67 kB 4.1 MB/s 
     |████████████████████████████████| 829 kB 35.8 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 180 kB 59.7 MB/s 
     |████████████████████████████████| 106 kB 65.3 MB/s 
     |████████████████████████████████| 175 kB 63.6 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 99 kB 8.6 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 130 kB 68.7 MB/s 
     |████████████████████████████████| 564 kB 55.6 MB/s 


In [ ]:
#Importing libraries
import pandas as pd 
import scipy.sparse as sps
import numpy as np
import time
import matplotlib.pyplot as pyplot
import os
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.Similarity.Compute_Similarity_Python import Compute_Similarity_Python
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender

print("DEBUG")

DEBUG


In [ ]:
%cd ..
%ls

/content/gdrive/MyDrive/RS_Challenge
 RSDs/   'RS_MFIals 41,8%.ipynb'             submission_MFIALS_full_URM.csv
 RSLib/   submission_MFIALS_2_full_URM.csv


In [ ]:
#Loading dataframes
URM_dataframe = pd.read_csv('RSDs/data_train.csv')
ICM_genre_dataframe=pd.read_csv('RSDs/data_ICM_genre.csv')
ICM_subgenre_dataframe=pd.read_csv('RSDs/data_ICM_subgenre.csv')
ICM_event_dataframe=pd.read_csv('RSDs/data_ICM_event.csv')
ICM_channel_dataframe=pd.read_csv('RSDs/data_ICM_channel.csv')

In [ ]:
#Column Renaming
URM_dataframe.columns = ['User','Item','Interaction']
ICM_genre_dataframe.columns = ['Item','Feature','Val']
ICM_subgenre_dataframe.columns = ['Item','Feature','Val']
ICM_event_dataframe.columns = ['Item','Feature','Val']
ICM_channel_dataframe.columns = ['Item','Feature','Val']

In [ ]:
#Converting datatype
URM_dataframe['Interaction'] = URM_dataframe['Interaction'].astype(int)
ICM_genre_dataframe['Val'] = ICM_genre_dataframe['Val'].astype(int)
ICM_subgenre_dataframe['Val'] = ICM_subgenre_dataframe['Val'].astype(int)
ICM_event_dataframe['Val'] = ICM_event_dataframe['Val'].astype(int)
ICM_channel_dataframe['Val'] = ICM_channel_dataframe['Val'].astype(int)

In [ ]:
#Printing stats to see if everything went well
userID_unique = URM_dataframe["User"].unique()
itemID_unique = URM_dataframe["Item"].unique()
n_users = len(userID_unique)
n_items = len(itemID_unique)
n_interactions = len(URM_dataframe)

#Printing some stats
print("Number of users: " + str(n_users))
print("Number of items: " + str(n_items))
print("Number of interactions: " + str(n_interactions))

Number of users: 13650
Number of items: 18059
Number of interactions: 5285664


In [ ]:
#Checking if re-assignment of ID's is needed
print('The maxID for user {}, The maxID for item {}'.format(max(URM_dataframe.User),max(URM_dataframe.Item)))

The maxID for user 13649, The maxID for item 18058


Re-assignment not needed

In [ ]:
#Converting the URM into a sparse matrix
URM_sparse = sps.coo_matrix((URM_dataframe['Interaction'].values,(URM_dataframe['User'].values,URM_dataframe['Item'].values)))
URM_csr = URM_sparse.tocsr()
URM_csr

<13650x18059 sparse matrix of type '<class 'numpy.longlong'>'
	with 5285664 stored elements in Compressed Sparse Row format>

In [ ]:
#Splitting data into training and validation set (Test set is the one on Kaggle)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_csr, train_percentage = 0.80)
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13642 ( 0.1%) Users that have less than 1 test interactions


In [ ]:
#Using MF IALS imported from course's libraries
recommender = IALSRecommender(URM_sparse) # Previously I've worked with train+validation, now I use the entire URM to obtain better performances on the test set

Hyperparameters tuning

In [ ]:
from skopt.space import Real, Integer, Categorical

In [ ]:
hyperparameters_range_dictionary = {
    "num_factors": Integer(45,55),
    "alpha": Real(0.95,1.05),
    "reg": Real(0.0005,0.0007),
    "init_mean": Real(0.4, 0.6),
    "init_mean": Real(0.04, 0.06),
    "confidence_scaling": Categorical(["linear"])
}

In [ ]:
recommender_class = IALSRecommender
hyperparameterSearch = SearchBayesianSkopt(recommender_class, evaluator_validation=evaluator_validation)

In [ ]:
output_folder_path = '/content/gdrive/MyDrive/Colab Notebooks/RSChallenge/Results/SLIM_res/'

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 10 
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [ ]:
earlystopping_keywargs = {"validation_every_n": 2,
                          "stop_on_validation": True,
                          "evaluator_object": evaluator_validation,
                          "lower_validations_allowed": 10,
                          "validation_metric": metric_to_optimize,
                          }

In [ ]:
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],   
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = earlystopping_keywargs # Early stopping
)

recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_validation],  
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = earlystopping_keywargs # Early stopping
)

In [ ]:
hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                         evaluator_validation=evaluator_validation)

In [ ]:
hyperparameterSearch.search(recommender_input_args,
                       recommender_input_args_last_test = recommender_input_args_last_test,
                       hyperparameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       save_model = "last",
                       output_folder_path = output_folder_path,
                       output_file_name_root = recommender_class.RECOMMENDER_NAME,
                       metric_to_optimize = metric_to_optimize,
                       cutoff_to_optimize = cutoff_to_optimize,
                      )

In [ ]:
recommender.fit(epochs = 32, num_factors = 50, alpha = 1.011955985422186, reg = 0.0006510527003718695, init_mean = 0.4902658701038708, init_std = 0.0542837791767091) # Così ottengo: 0.418 MAP: 0.2335376

IALSRecommender: Epoch 1 of 32. Elapsed time 15.90 sec
IALSRecommender: Epoch 2 of 32. Elapsed time 32.10 sec
IALSRecommender: Epoch 3 of 32. Elapsed time 48.22 sec
IALSRecommender: Epoch 4 of 32. Elapsed time 1.20 min
IALSRecommender: Epoch 5 of 32. Elapsed time 1.60 min
IALSRecommender: Epoch 6 of 32. Elapsed time 1.92 min
IALSRecommender: Epoch 7 of 32. Elapsed time 2.19 min
IALSRecommender: Epoch 8 of 32. Elapsed time 2.45 min
IALSRecommender: Epoch 9 of 32. Elapsed time 2.72 min
IALSRecommender: Epoch 10 of 32. Elapsed time 3.00 min
IALSRecommender: Epoch 11 of 32. Elapsed time 3.26 min
IALSRecommender: Epoch 12 of 32. Elapsed time 3.53 min
IALSRecommender: Epoch 13 of 32. Elapsed time 3.80 min
IALSRecommender: Epoch 14 of 32. Elapsed time 4.07 min
IALSRecommender: Epoch 15 of 32. Elapsed time 4.34 min
IALSRecommender: Epoch 16 of 32. Elapsed time 4.60 min
IALSRecommender: Epoch 17 of 32. Elapsed time 4.87 min
IALSRecommender: Epoch 18 of 32. Elapsed time 5.14 min
IALSRecommender:

In [ ]:
%ls

 RSDs/   RSLib/  'RS_MF_IALS 41,8%.ipynb'   submission_MFIALS_2_full_URM.csv


In [ ]:
target_users = pd.read_csv('RSDs/data_target_users_test.csv')

In [ ]:
final_dict = {}
start_time = time.time()

for user in target_users['user_id']:
  final_dict[user] = recommender.recommend(user,cutoff=10)
end_time = time.time()
print('Time needed is {}seconds'.format(end_time-start_time))

Time needed is 19.083093881607056seconds


In [ ]:
item_list = []
for user in target_users['user_id']:
  item_list.append(' '.join(str(e) for e in final_dict[user]))

In [ ]:
submission_df = pd.DataFrame({'user_id' : target_users['user_id'],'item_list' : item_list })
submission_df['item_list'] = submission_df['item_list'].astype(str).str.strip('[]')
submission_df

,user_id,item_list
0,0,3899 6061 12752 8635 16359 5433 5885 5981 7143...
1,1,940 3473 2118 9275 3402 2820 53 739 16365 1533
2,2,6177 14181 1551 17330 5273 14229 15119 17584 5...
3,3,8194 960 15082 15551 15844 5885 5981 16359 837...
4,4,8194 11836 9871 7870 9825 13691 10586 241 9573...
...,...,...
13645,13645,6177 6961 16332 11491 4078 3200 9527 840 1259 ...
13646,13646,3473 5324 16640 7867 17266 8635 16624 17623 12...
13647,13647,1846 7716 7606 11227 14868 2665 8635 11491 173...
13648,13648,6177 8635 5273 13134 6214 14181 1846 11058 989...


In [ ]:
%cd ..
%ls

/content/gdrive/MyDrive
'Anthony super bello 2.zip'               'LISTA EPISODI COMPLETA 7.4.gdoc'
 Backup_Rubrica.vcf                       'Modulo senza titolo.gform'
 Cattura.JPG                              'PROGETTO DEF NON OTTIMIZZATO.txt'
'CENSORED TRACKLIST.png'                  'PROGETTO DEF OTTIMIZZATO.txt'
'CENSORED TRACKLIST STORIA.png'            RS_Challenge/
'Colab Notebooks'/                         SantoriniClient.zip
 content/                                 'SONGS REVIEW'/
'DRIVER EPSON'/                           'TRACKLIST CLEAN.png'
'FSM RESTAR MULTI START OTTIMIZZATA.txt'  'TRACKLIST CLEAN STORY.png'


In [ ]:
submission_df.to_csv('RS_Challenge/submission_MFIALS_full_URM.csv', index=False)

Loading a DF and working with it

In [ ]:
%cd ..
%ls

/content/gdrive/MyDrive
'Anthony super bello 2.zip'               'LISTA EPISODI COMPLETA 7.4.gdoc'
 Backup_Rubrica.vcf                       'Modulo senza titolo.gform'
 Cattura.JPG                              'PROGETTO DEF NON OTTIMIZZATO.txt'
'CENSORED TRACKLIST.png'                  'PROGETTO DEF OTTIMIZZATO.txt'
'CENSORED TRACKLIST STORIA.png'            RS_Challenge/
'Colab Notebooks'/                         SantoriniClient.zip
 content/                                 'SONGS REVIEW'/
'DRIVER EPSON'/                           'TRACKLIST CLEAN.png'
'FSM RESTAR MULTI START OTTIMIZZATA.txt'  'TRACKLIST CLEAN STORY.png'


In [ ]:
MF_rec = pd.read_csv('RS_Challenge/submission_MFIALS_full_URM.csv') # Loading DF
SLIM_rec = pd.read_csv('RS_Challenge/submission_SLIM_full_URM.csv') # Loading DF

In [ ]:
MF_rec # Printing DF

,user_id,item_list
0,0,3899 6061 12752 8635 16359 5433 5885 5981 7143...
1,1,940 3473 2118 9275 3402 2820 53 739 16365 1533
2,2,6177 14181 1551 17330 5273 14229 15119 17584 5...
3,3,8194 960 15082 15551 15844 5885 5981 16359 837...
4,4,8194 11836 9871 7870 9825 13691 10586 241 9573...
...,...,...
13645,13645,6177 6961 16332 11491 4078 3200 9527 840 1259 ...
13646,13646,3473 5324 16640 7867 17266 8635 16624 17623 12...
13647,13647,1846 7716 7606 11227 14868 2665 8635 11491 173...
13648,13648,6177 8635 5273 13134 6214 14181 1846 11058 989...


In [ ]:
SLIM_rec

,user_id,item_list
0,0,3899 5433 8465 16365 16359 2243 9130 6061 1058...
1,1,3473 940 14400 16012 2118 10118 9275 9373 640 ...
2,2,6177 1551 17330 17022 2846 15119 14229 5925 16...
3,3,8194 15551 16359 15082 8371 960 15507 16444 16...
4,4,8194 10586 6177 16494 2152 5299 9130 10954 347...
...,...,...
13645,13645,6177 11491 3200 8898 11576 11303 1259 562 7762...
13646,13646,3473 7867 8635 16640 409 11504 17266 17622 176...
13647,13647,1846 7716 11227 7606 1186 8635 14868 11491 168...
13648,13648,6177 8635 13295 7295 14358 14181 6214 2482 179...


In [ ]:
# Reading the DF line by line

MF_ll = MF_rec.values.tolist() # Converting the DF in a List of Lists
MF_list = MF_ll[0][1].split() # Taking the recommendations for user 0 and coverting them into a list
MF_list[3] # Taking the 3th elements recommended to user 0

'8635'

In [ ]:
%ls

'Anthony super bello 2.zip'               'LISTA EPISODI COMPLETA 7.4.gdoc'
 Backup_Rubrica.vcf                       'Modulo senza titolo.gform'
 Cattura.JPG                              'PROGETTO DEF NON OTTIMIZZATO.txt'
'CENSORED TRACKLIST.png'                  'PROGETTO DEF OTTIMIZZATO.txt'
'CENSORED TRACKLIST STORIA.png'            RS_Challenge/
'Colab Notebooks'/                         SantoriniClient.zip
 content/                                 'SONGS REVIEW'/
'DRIVER EPSON'/                           'TRACKLIST CLEAN.png'
'FSM RESTAR MULTI START OTTIMIZZATA.txt'  'TRACKLIST CLEAN STORY.png'


In [ ]:
target_users = pd.read_csv('RS_Challenge/RSDs/data_target_users_test.csv')

In [ ]:
user_id = 1 
i = 0 # Per scorrere SLIM
j = 0 # Per scorrere MF
  
list_temp = []

SLIM_ll = SLIM_rec.values.tolist()
MF_ll = MF_rec.values.tolist()

MF_list = MF_ll[user_id][1].split()
SLIM_list = SLIM_ll[user_id][1].split()

print("MF ")
print(MF_list)
print("SLIM ")
print(SLIM_list)

for k in range(10):
  if(SLIM_list[k] in MF_list):
    list_temp.append(SLIM_list[k])

print("Common")
print(list_temp)    

while(len(list_temp)<10):
  if(i==2*j): # Se è il turno di pescare da SLIM
    if(SLIM_list[i] not in list_temp):
      list_temp.append(SLIM_list[i])
    i=i+1
    if(SLIM_list[i] not in list_temp and len(list_temp)<10):
      list_temp.append(SLIM_list[i])
    i=i+1
  else: # Se è il turno di pescare da MF
    if(MF_list[j] not in list_temp):
      list_temp.append(MF_list[j])
    j=j+1

print("Suggestions")
print(list_temp)
  


MF 
['940', '3473', '2118', '9275', '3402', '2820', '53', '739', '16365', '1533']
SLIM 
['3473', '940', '14400', '16012', '2118', '10118', '9275', '9373', '640', '16365']
Common
['3473', '940', '2118', '9275', '16365']
Suggestions
['3473', '940', '2118', '9275', '16365', '14400', '16012', '10118', '9373', '640']


In [ ]:
final_dict = {}
start_time = time.time()

SLIM_ll = SLIM_rec.values.tolist()
MF_ll = MF_rec.values.tolist()

for user in target_users['user_id']:
  i = 0 # Per scorrere SLIM
  j = 0 # Per scorrere MF

  list_temp = [] # Suggestions

  MF_list = MF_ll[user][1].split()
  SLIM_list = SLIM_ll[user][1].split()

  for k in range(10):
    if(SLIM_list[k] in MF_list):
      list_temp.append(SLIM_list[k])

  while(len(list_temp)<10):
    if(i==3*j): # Se è il turno di pescare da SLIM (2)
      if(SLIM_list[i] not in list_temp):
        list_temp.append(SLIM_list[i])
      i=i+1
      if(i<10 and SLIM_list[i] not in list_temp and len(list_temp)<10):
        list_temp.append(SLIM_list[i])
      i=i+1
      if(i<10 and SLIM_list[i] not in list_temp and len(list_temp)<10):
        list_temp.append(SLIM_list[i])
      i=i+1
    else: # Se è il turno di pescare da MF (1)
      if(MF_list[j] not in list_temp):
        list_temp.append(MF_list[j])
      j=j+1

  final_dict[user] = list_temp

end_time = time.time()
print('Time needed is {}seconds'.format(end_time-start_time))

Time needed is 0.37792468070983887seconds


In [ ]:
item_list = []
for user in target_users['user_id']:
  item_list.append(' '.join(str(e) for e in final_dict[user]))

In [ ]:
submission_df = pd.DataFrame({'user_id' : target_users['user_id'],'item_list' : item_list })
submission_df['item_list'] = submission_df['item_list'].astype(str).str.strip('[]')
submission_df

,user_id,item_list
0,0,3899 5433 16359 6061 640 8465 16365 2243 9130 ...
1,1,3473 940 2118 9275 16365 14400 16012 10118 937...
2,2,6177 1551 17330 15119 14229 17022 2846 14181 5...
3,3,8194 15551 16359 15082 8371 960 5981 15507 164...
4,4,8194 10586 6177 16494 2152 5299 11836 9130 109...
...,...,...
13645,13645,6177 11491 3200 1259 8898 11576 11303 6961 562...
13646,13646,3473 7867 8635 16640 17266 17623 409 11504 532...
13647,13647,1846 7716 11227 7606 8635 14868 11491 16846 11...
13648,13648,6177 8635 14181 6214 1846 13295 7295 14358 248...


In [ ]:
%cd MyDrive
%ls

[Errno 2] No such file or directory: 'MyDrive'
/content/gdrive/MyDrive
'Anthony super bello 2.zip'               'LISTA EPISODI COMPLETA 7.4.gdoc'
 Backup_Rubrica.vcf                       'Modulo senza titolo.gform'
 Cattura.JPG                              'PROGETTO DEF NON OTTIMIZZATO.txt'
'CENSORED TRACKLIST.png'                  'PROGETTO DEF OTTIMIZZATO.txt'
'CENSORED TRACKLIST STORIA.png'            RS_Challenge/
'Colab Notebooks'/                         SantoriniClient.zip
 content/                                 'SONGS REVIEW'/
'DRIVER EPSON'/                           'TRACKLIST CLEAN.png'
'FSM RESTAR MULTI START OTTIMIZZATA.txt'  'TRACKLIST CLEAN STORY.png'


In [ ]:
submission_df.to_csv('RS_Challenge/submission_Round_Robin_3SLIM1MF_Common.csv', index=False)

Check how much my hybrid is different from SLIM

In [ ]:
Hybrid = pd.read_csv('RS_Challenge/submission_Round_Robin_3SLIM1MF_Common.csv')
Hybrid_ll = Hybrid.values.tolist() # Converting the DF in a List of Lists
Hybrid_list = Hybrid_ll[0][1].split() # Taking the recommendations for user 0 and coverting them into a list
print(Hybrid_list)
print(len(Hybrid_list))

['3899', '5433', '16359', '6061', '640', '8465', '16365', '2243', '9130', '10586']
10


In [ ]:
# Controlliamo che nell'hybrid ci siano 10 oggetti per utente

len_ok = True

for user in target_users['user_id']:
  Hybrid_list = Hybrid_ll[user][1].split()
  if len(Hybrid_list) != 10:
    len_ok = False

if(len_ok):
  print("OK")


OK


In [ ]:
# Controlliamo quanto l'hybrid appena creato è diverso da SLIM (Hybrid con 3SLIM 1MF)

num10 = 0
num9 = 0
num8 = 0
num7 = 0
num6 = 0
num5 = 0
num4 = 0
num3 = 0
num2 = 0
num1 = 0
num0 = 0

for user in target_users['user_id']:
  count = 0
  Hybrid_list = Hybrid_ll[user][1].split()
  SLIM_list = SLIM_ll[user][1].split()
  for k in range(10):
    if(SLIM_list[k] in Hybrid_list):
      count = count + 1
  
  if count == 10: num10=num10+1
  if count == 9: num9=num9+1
  if count == 8: num8=num8+1
  if count == 7: num7=num7+1
  if count == 6: num6=num6+1
  if count == 5: num5=num5+1
  if count == 4: num4=num4+1
  if count == 3: num3=num3+1
  if count == 2: num2=num2+1
  if count == 1: num1=num1+1

print(num10)
print(num9)
print(num8)
print(num7)
print(num6)
print(num5)
print(num4)
print(num3)
print(num2)
print(num1)

9078
4143
429
0
0
0
0
0
0
0


In [ ]:
Hybrid = pd.read_csv('RS_Challenge/submission_Round_Robin_2SLIM1MF_Common.csv')
Hybrid_ll = Hybrid.values.tolist() # Converting the DF in a List of Lists
Hybrid_list = Hybrid_ll[0][1].split() # Taking the recommendations for user 0 and coverting them into a list
print(Hybrid_list)
print(len(Hybrid_list))

['3899', '5433', '16359', '6061', '640', '8465', '16365', '2243', '12752', '9130']
10


In [ ]:
# Controlliamo quanto l'hybrid appena creato è diverso da SLIM (Hybrid con 2SLIM 1MF)

num10 = 0
num9 = 0
num8 = 0
num7 = 0
num6 = 0
num5 = 0
num4 = 0
num3 = 0
num2 = 0
num1 = 0
num0 = 0

for user in target_users['user_id']:
  count = 0
  Hybrid_list = Hybrid_ll[user][1].split()
  SLIM_list = SLIM_ll[user][1].split()
  for k in range(10):
    if(SLIM_list[k] in Hybrid_list):
      count = count + 1
  
  if count == 10: num10=num10+1
  if count == 9: num9=num9+1
  if count == 8: num8=num8+1
  if count == 7: num7=num7+1
  if count == 6: num6=num6+1
  if count == 5: num5=num5+1
  if count == 4: num4=num4+1
  if count == 3: num3=num3+1
  if count == 2: num2=num2+1
  if count == 1: num1=num1+1

print(num10)
print(num9)
print(num8)
print(num7)
print(num6)
print(num5)
print(num4)
print(num3)
print(num2)
print(num1)

6617
5068
1810
155
0
0
0
0
0
0
